# TP3 - Views, Updates and Design

The objectives for this TP are:

1. Create and use SQL Views
2. Update database content
3. Design the database schema for a Social Network

___

For the first 2 parts we will use the **`wine.db`** database and the Tables created in TP2.

A reminder of the wine database schema:

<center>**Master1**</center>

|*Attribute*|         *Description*          |
| -------   |--------------------------------|
| NV        | Wine number                    |
| CRU       | Vineyard or group of vineyards |
| DEGRE     | Alcohol content                |
| MILL      | Vintage year                   |
| QTE       | Number of bottles harvested    |
| NP        | Producer number                |
| NOM       | Producer's last name           |
| PRENOM    | Producer's first name          |
| REGION    | Production region              |

<center>**Master2**</center>

|*Attribute*|                         *Description*                  |
| -------   |--------------------------------------------------------|
| NV        | Wine number                                            |
| CRU       | Vineyard or group of vineyards                         |
| DEGRE     | Alcohol content                                        |
| MILL      | Vintage year                                           |
| DATES     | Buying date                                            |
| LIEU      | Place where the wine was sold                          |
| QTE       | Number of bottles bought                               |
| NB        | Client (buveur) number                                 |
| NOM       | Client's last name                                     |
| PRENOM    | Client's first name                                    |
| TYPE      | Type of client by volume of purchases                  |
| REGION    | Administrative Region (different to production region) |


In [1]:
import sqlite3

In [2]:
def printSchema(connection):
    ### Source: http://stackoverflow.com/a/35092773/4765776
    for (tableName,) in connection.execute(
        """
        select NAME from SQLITE_MASTER where TYPE='table' order by NAME;
        """
    ):
        print("{}:".format(tableName))
        for (
            columnID, columnName, columnType,
            columnNotNull, columnDefault, columnPK,
        ) in connection.execute("pragma table_info('{}');".format(tableName)):
            print("  {id}: {name}({type}){null}{default}{pk}".format(
                id=columnID,
                name=columnName,
                type=columnType,
                null=" not null" if columnNotNull else "",
                default=" [{}]".format(columnDefault) if columnDefault else "",
                pk=" *{}".format(columnPK) if columnPK else "",
            ))

In [4]:
conn = sqlite3.connect('wine.db')
c = conn.cursor()
print("Database schema:")
printSchema(conn)

Database schema:
Buveur:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
Localisation:
  0: LIEU(TEXT)
  1: REGION(TEXT)
MASTER1:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: QTE(NUM)
  5: NP(NUM)
  6: NOM(TEXT)
  7: PRENOM(TEXT)
  8: REGION(TEXT)
MASTER2:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
  4: DATES(DATE)
  5: LIEU(TEXT)
  6: QTE(NUM)
  7: NB(NUM)
  8: NOM(TEXT)
  9: PRENOM(TEXT)
  10: TYPE(TEXT)
  11: REGION(TEXT)
Producteur:
  0: NOM(TEXT)
  1: PRENOM(TEXT)
  2: REGION(TEXT)
  3: NP(NUM)
Production:
  0: NP(NUM)
  1: NV(NUM)
  2: QTE(NUM)
RBB:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: TYPE(TEXT)
S:
  0: NB(NUM)
  1: NOM(TEXT)
  2: PRENOM(TEXT)
  3: SOMME()
Vente:
  0: NV(NUM)
  1: NB(NUM)
  2: DATES(NUM)
  3: LIEU(TEXT)
  4: QTE(NUM)
Vin:
  0: NV(NUM)
  1: CRU(TEXT)
  2: DEGRE(NUM)
  3: MILL(NUM)
Wine:
  0: CRU(TEXT)
  1: DEGRE(NUM)
  2: MILL(NUM)
  3: NV(NUM)
dummy:
  0: DEGRE(NUM)


Again, we will use **`%%sql`** magic for our queries

In [5]:
%load_ext sql
%sql sqlite:///wine.db

'Connected: @wine.db'

Recreate the Normalized Tables from **Master1** and **Master2** as you did in the TP2

In [6]:
%%sql 
DROP TABLE IF EXISTS Vin;
CREATE TABLE Vin AS 
SELECT DISTINCT CRU,DEGRE,MILL,NV
FROM Master1 
WHERE NV IS NOT NULL;

DROP TABLE IF EXISTS Producteur;
CREATE TABLE Producteur AS 
SELECT DISTINCT NOM,PRENOM,REGION,NP
FROM Master1 
WHERE NP IS NOT NULL;

DROP TABLE IF EXISTS Production;
CREATE TABLE Production AS 
SELECT DISTINCT NP,NV,QTE
FROM Master1 
WHERE NP IS NOT NULL;

DROP TABLE IF EXISTS Buveur;
CREATE TABLE Buveur AS 
SELECT DISTINCT NB,NOM,PRENOM,TYPE
FROM Master2 
WHERE NB IS NOT NULL;

DROP TABLE IF EXISTS Vente;
CREATE TABLE Vente AS 
SELECT DISTINCT NV,NB,DATES,LIEU,QTE
FROM Master2 
WHERE NV IS NOT NULL AND NB IS NOT NULL;

DROP TABLE IF EXISTS Vin;
CREATE TABLE Vin AS 
SELECT DISTINCT NV,CRU,DEGRE,MILL
FROM Master2
WHERE (NV IS NOT NULL) ;

DROP TABLE IF EXISTS Localisation;
CREATE TABLE Localisation AS 
SELECT DISTINCT LIEU,REGION
FROM Master2 
WHERE LIEU IS NOT NULL;




 * sqlite:///wine.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


NV,NB,DATES,LIEU,QTE
1,2,1977-11-02,BORDEAUX,33
1,44,2015-10-16,PARIS,1
1,45,1983-12-31,RENNES,1
2,48,1983-12-25,LYON,2
3,7,1978-11-01,NICE,6
3,49,1987-12-24,MERCUREY,5
4,8,1982-11-05,TOULOUSE,12
4,44,2015-10-16,PARIS,6
4,50,1981-06-29,SENS,4
5,2,1984-11-04,MERCUREY,50


___
# PART I: CREATE AND USE VIEWS

A view is a virtual table based on the result-set of an SQL statement. Views are stored in the database with an associated name.

Views are created following the syntax:

```mysql
CREATE VIEW view_name AS
SELECT column1, column2.....
FROM table_name
WHERE [condition];
```

An useful command:

```mysql
DROP VIEW IF EXISTS view_name;
```


__Warning:__ Use `DROP` with caution (only drop something if you are sure)

__Note:__ You will find some cells marked as "Test" that will help you check your work. Do NOT modify them. 

#### Exercise 1.1

Create a view "**bons_buveurs**" with the clients (buveurs) of type 'gros' or 'moyen'.

In [9]:
%%sql
DROP VIEW IF EXISTS bon_buveurs;
CREATE VIEW bon_buveurs AS
SELECT NB,NOM,PRENOM,TYPE
FROM Buveur
WHERE (TYPE='gros' OR TYPE='moyen');
SELECT*FROM bon_buveurs;

 * sqlite:///wine.db
Done.
Done.
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


In [15]:
# Test
%sql SELECT * FROM bon_buveurs ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 1.2

Create the view "**buveurs_asec**" with clients (buveurs) who have not bought any wine.

In [16]:
%%sql
DROP VIEW IF EXISTS buveurs_asec;
CREATE VIEW buveurs_asec AS
SELECT NB,NOM,PRENOM
FROM MASTER2
WHERE NB NOT IN (SELECT NB FROM Vente);

 * sqlite:///wine.db
Done.
Done.


[]

In [38]:
# Test
%sql SELECT * FROM buveurs_asec ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
11,Breton,Andre
13,Barthes,Roland
16,Balzac,Honore de
18,Celine,Louis Ferdinand
20,Chateaubriand,Francois-Rene de
21,Corbiere,Tristan
23,Corneille,Pierre
25,Char,Rene
27,Dumas,Alexandre
29,Fournier,Alain


#### Exercise 1.3

Create the view "**buveurs_achats**" complementary to the previous one.

In [17]:
%%sql 
DROP VIEW IF EXISTS buveurs_achats;
CREATE VIEW buveurs_achats AS
SELECT DISTINCT Buveur.NB,NOM,PRENOM FROM (Vente JOIN Buveur on Vente.NB=Buveur.NB)


 * sqlite:///wine.db
Done.
Done.


[]

In [18]:
# Test
%sql SELECT * FROM buveurs_achats ORDER BY nb;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
1,Aristote,None
2,Artaud,Antonin
3,Aron,Raymond
4,Apollinaire,Guillaume
5,Audiberti,Jacques
6,Arrabal,Fernando
7,Anouilh,Jean
8,Aragon,Louis
9,Ajar,Emile
10,Andersen,Yann


#### Exercise 1.4

Create the view "**q83pl**" (LIEU, CRU, QTE_BUE) that provides by LIEU and CRU the total quantities bought in 1983 by all the clients (buveurs).

In [19]:
%%sql

DROP VIEW IF EXISTS q83pl;

CREATE VIEW q83pl AS
SELECT LIEU, CRU, SUM(QTE) AS QTE_BUE
FROM Vente JOIN Vin ON Vente.NV = Vin.NV
WHERE DATES LIKE "1983%"
GROUP BY LIEU, CRU;



 * sqlite:///wine.db
Done.
Done.


[]

In [20]:
# Test
%sql SELECT * FROM q83pl;


 * sqlite:///wine.db
Done.


LIEU,CRU,QTE_BUE
CAEN,Seyssel,3
LILLE,Pommard,5
LYON,Beaujolais Villages,10
LYON,Julienas,2
PARIS,Beaujolais Primeur,4
PARIS,Coteaux du Tricastin,1
PARIS,Pouilly Vinzelles,3
RENNES,Mercurey,1
ROCQUENCOURT,Beaujolais Villages,260
ROCQUENCOURT,Saint Amour,80


#### Exercise 1.5

Can we define the same view with ascending order over the attribute "QTE"? Provide an explanation for your answer.

___
# PART II: UPDATE DATABASE CONTENT

The syntax for the `UPDATE` operation is:

```sql
UPDATE table_name
SET column1 = value1, column2 = value2...., columnN = valueN
WHERE [condition];
```

The syntax for the `INSERT` operation is:

```sql
INSERT INTO table_name [(column1, column2, column3,...columnN)]  
VALUES (value1, value2, value3,...valueN);
```

Database updates are commited automatically in Jupyter/Python. _Transactions_ are an option to control and reverse changes. Additionally we can just reload a backup of the database (NOT an option in deployed systems)

__Note:__ Different to other Database Management Systems, SQLite views are read-only and so you can not execute a `DELETE`, `INSERT` or `UPDATE` statement on a view.

#### Exercise 2.1

Create a table "**RBB**" with the same schema as "**bons_buveurs**" which contains the tuples selected from "**bons_buveurs**"

In [21]:
%%sql
DROP TABLE IF EXISTS RBB;
CREATE TABLE RBB AS
SELECT *FROM bon_buveurs


 * sqlite:///wine.db
Done.
Done.


[]

In [22]:
# Test
%sql SELECT * FROM RBB ORDER BY NB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
2,Artaud,Antonin,moyen
3,Aron,Raymond,gros
4,Apollinaire,Guillaume,moyen
6,Arrabal,Fernando,gros
7,Anouilh,Jean,moyen
8,Aragon,Louis,gros
10,Andersen,Yann,gros
12,Bataille,Georges,moyen
13,Barthes,Roland,moyen
14,Bory,Jean Louis,gros


#### Exercise 2.2

Update the table you used to create "**bons_buveurs**": Change the "type" to 'gros' if the total of quantities bought is over 100.

Find the instances to update (schema may differ from the one in your table)

In [33]:
%%sql 
UPDATE Buveur
SET TYPE='gros'
WHERE NB IN (
SELECT Vente.NB
FROM Vente NATURAL JOIN Buveur
WHERE TYPE!='gros'
GROUP BY Vente.NB
HAVING (SUM(QTE) >100 ))






 * sqlite:///wine.db
4 rows affected.


[]

Update instances

#### Exercise 2.3

Compare the content of _table_ "**RBB**" and the _view_ "**bons_buveurs**" after the update. What differences do you see? Explain

In [37]:
%%sql 
SELECT*
FROM bon_buveurs

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


In [36]:
%%sql 
SELECT*
FROM RBB;

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM,TYPE
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
27,Dumas,Alexandre,gros
32,Eluard,Paul,moyen
35,Fromentin,Eugene,gros
39,Montesquieu,None,gros
42,Goethe,Johann Wolfgang von,moyen
43,Musset,Alfred de,gros


#### Exercise 2.4

Create a table "**RBA**" with the same schema as "**buveurs_asec**" which contains the tuples selected from "**buveurs_asec**"

In [40]:
%%sql
DROP TABLE IF EXISTS RBA;
CREATE TABLE RBA AS
SELECT NB,NOM,PRENOM
FROM MASTER2
WHERE NB NOT IN (SELECT NB FROM Vente);



 * sqlite:///wine.db
Done.
Done.


[]

In [19]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.5

Insert a tuple (101, 'your last name', 'your first name', 'your type of purchases(petit, moyen, or gros)') to "**RBA**".

In [43]:
%%sql
INSERT INTO RBA (NB, NOM, PRENOM)
VALUES (101,'Habis','Antoine');

 * sqlite:///wine.db
1 rows affected.


[]

In [21]:
# Test
%sql SELECT * FROM RBA

Done.


NB,NOM,PRENOM,TYPE
11,Breton,Andre,petit
13,Barthes,Roland,moyen
16,Balzac,Honore de,moyen
18,Celine,Louis Ferdinand,gros
20,Chateaubriand,Francois-Rene de,moyen
21,Corbiere,Tristan,petit
23,Corneille,Pierre,petit
25,Char,Rene,petit
27,Dumas,Alexandre,gros
29,Fournier,Alain,petit


#### Exercise 2.6

Compare the content of _table_ "**RBA**" and the _view_ "**buveurs_asec**". What differences do you see? Explain

In [44]:
%%sql
SELECT*FROM RBA

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
11,Breton,Andre
13,Barthes,Roland
16,Balzac,Honore de
18,Celine,Louis Ferdinand
20,Chateaubriand,Francois-Rene de
21,Corbiere,Tristan
23,Corneille,Pierre
25,Char,Rene
27,Dumas,Alexandre
29,Fournier,Alain


In [45]:
%%sql
SELECT* FROM buveurs_asec

 * sqlite:///wine.db
Done.


NB,NOM,PRENOM
11,Breton,Andre
13,Barthes,Roland
16,Balzac,Honore de
18,Celine,Louis Ferdinand
20,Chateaubriand,Francois-Rene de
21,Corbiere,Tristan
23,Corneille,Pierre
25,Char,Rene
27,Dumas,Alexandre
29,Fournier,Alain


___
# PART III: Design the database schema for posts in a Social Network

In this section your task is to design the database schema for a social network app of a new startup:

The new social network will contain users, where each user will have a name, a nickname, an email, date of birth, and an address (Street, City, State, Country, Postal Code). Users can be friends of other users, and can publish posts. Each post can contain a text, date and attachment. Posts can be either original posts or replies so the app needs to handle both scenarios. When users log in, the app needs to display the posts of their friends.

**Note:** You can create diagrams of your proposal and insert them as images into this notebook.

#### Exercise 3.1

Write and explain the design of the relations of your database

#### Exercise 3.2

Write a view to retrieve the posts to display when a user logs in. Consider that some users may have a lot of friends and you need to limit the number of post to display. How would you select relevant posts to display first? What kind of information would you use/add in the database for this purpose? Explain your answer.

__Note:__ Limiting the number of posts just by count is too simplistic, the user could be missing something interesting to him/her.